In [60]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np
import pandas as pd

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
def csvAdapter(dataset,modificarColumnas,eliminarColumnas):
    data =pd.read_csv(dataset,encoding='latin-1')#lee y carga el archivo del data set a la variable data
    columna = data.columns #asignamos los nombres de columna a la variables columna

    for index in modificarColumnas:
        name = columna[index]
        value = list(data[name].unique())#obetenemos todos lo valores unicos de cada columna
        data[name] = data[name].map(dict(zip(value,[i for i in range(len(value))]))) #aqui es donde hacemos el tratamiento de datos donde a cada valor unico aiganmos un indice, indice que es numerico ejm 0=femenino, 1=masculino  
    data = data.drop([columna[index]for index in eliminarColumnas],axis=1)#eliminamos las columnas que no interesan en la resolucion o prediccion del problema   
    data = data.dropna()#eliminamos las filas que contengan valores nulos
    #data = pd.DataFrame(data, columns=columna)
    #dataN = data.to_numpy()
    print(data.head())
    return data
rutadataset='/content/drive/MyDrive/DataSets/seattle-weather.csv'
columnaCategoricas = [i for i in range(0, 6)]# poner columnas que no son categoricas o no interesan enla prediccionn
eliminarColumnas=[0]#columnas a eliminar
data = csvAdapter(rutadataset, columnaCategoricas, eliminarColumnas)

   precipitation  temp_max  temp_min  wind  weather
0              0         0         0     0        0
1              1         1         1     1        1
2              2         2         2     2        1
3              3         3         3     0        1
4              4         4         1     3        1


In [63]:
def DatosSinteticos(data):
    n_Filas = 10000
    datosNuevos = np.empty([n_Filas, len(data.columns)])#creamos una matriz vacia con 10000 fila y las cantidad de columnas existentes en el dataset
    
    for i in range(len(data.columns)):
        columna_i = data.iloc[:,i]
        muestras_i = np.random.choice(columna_i,n_Filas, replace=True)#generar los numeros aleatorio dependiendo de la muetra tomada, replace 
        datosNuevos[:,i] = muestras_i.round()#redondeamos las muestras para trabajar de mejor manera                           #replace indica si las muestras pueden o no repetirse 
    columnas = list(data.columns) #crea una lista de los nombres de columnas
    datosNuevos_df = pd.DataFrame(datosNuevos, columns=columnas) #crea un nuevo data frame 

    dataExtend = pd.concat([data, datosNuevos_df], axis=0)#concatena el data con los datos sinteticos creados
    print(dataExtend.tail())
    return dataExtend

data = DatosSinteticos(data)
data.size

      precipitation  temp_max  temp_min  wind  weather
9995            0.0      17.0      16.0  25.0      1.0
9996            0.0       3.0      16.0   5.0      1.0
9997            0.0      17.0       5.0  26.0      1.0
9998            0.0       8.0      26.0  49.0      4.0
9999           25.0      47.0      16.0  14.0      2.0


57305

In [64]:
num_labels = 4
data = data.to_numpy()
X, y = data[:7000, 0:4], data[:7000, 4]
m = y.size

In [65]:
print(X)
print(y)

[[ 0.  0.  0.  0.]
 [ 1.  1.  1.  1.]
 [ 2.  2.  2.  2.]
 ...
 [ 0.  4. 16.  6.]
 [15.  6. 27. 19.]
 [49. 19. 24. 22.]]
[0. 1. 1. ... 2. 1. 1.]


In [66]:
def displayData(X, example_width=None, figsize=(10, 10)):
    """
    Muestra datos 2D almacenados en X en una cuadrícula apropiada.
    """
    # Calcula filas, columnas
    if X.ndim == 2:
        m, n = X.shape
    elif X.ndim == 1:
        n = X.size
        m = 1
        X = X[None]  # Promocionar a una matriz bidimensional
    else:
        raise IndexError('La entrada X debe ser 1 o 2 dimensinal.')

    example_width = example_width or int(np.round(np.sqrt(n)))
    example_height = n / example_width

    # Calcula el numero de elementos a mostrar
    display_rows = int(np.floor(np.sqrt(m)))
    display_cols = int(np.ceil(m / display_rows))

    fig, ax_array = pyplot.subplots(display_rows, display_cols, figsize=figsize)
    fig.subplots_adjust(wspace=0.025, hspace=0.025)

    ax_array = [ax_array] if m == 1 else ax_array.ravel()

    for i, ax in enumerate(ax_array):
        ax.imshow(X[i].reshape(example_width, example_width, order='F'),
                  cmap='Greys', extent=[0, 1, 0, 1])
        ax.axis('off')

In [67]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate([np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0], axis=1)
print(X_t)
# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 3

[[1.  0.1 0.6 1.1]
 [1.  0.2 0.7 1.2]
 [1.  0.3 0.8 1.3]
 [1.  0.4 0.9 1.4]
 [1.  0.5 1.  1.5]]


In [68]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [69]:
h = sigmoid(0)
print(h)

0.5


In [70]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y 
    el gradiente del costo w.r.t. a los parámetros.
    
    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas 
        incluida la intercepcion
        
    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de 
        caracteristicas (incluida la intercepcion).
    
    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos
    
    lambda_ : float
        Parametro de regularización. 
    
    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada. 
    
    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la 
        función de costo con respecto a theta, en los valores actuales de theta..
    """
    # Inicializa algunos valores utiles
    m = y.size
    
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X) 
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [71]:
J, grad = lrCostFunction(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 2.534819
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [0.146561, -0.548558, 0.724722, 1.398003]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


In [87]:
def oneVsAll(X, y, num_labels, lambda_):
    # algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 100}
        res = optimize.minimize(lrCostFunction, 
                                initial_theta, 
                                (X, (y == c), lambda_), 
                                jac=True, 
                                method='CG',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [89]:
lambda_ = 0.001
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)
all_theta

(4, 5)


array([[-3.25125202e+00, -9.55571192e-03,  6.66204896e-03,
        -3.87545061e-03,  1.18359204e-03],
       [-1.96042675e-01,  1.12338236e-02, -6.86176567e-03,
        -5.96284800e-03,  1.74245957e-03],
       [-4.14071830e-01, -8.99559692e-03,  8.02146021e-03,
         8.43214066e-03, -2.12358780e-03],
       [-3.21606032e+00,  2.41620081e-03, -1.81769364e-02,
        -2.29045503e-02, -2.66732625e-03]])

In [90]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [96]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[5000:5001, :].copy()
print(XPrueba.shape)
print(XPrueba)
XPrueba = np.concatenate([np.ones((1, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)




(7000, 4)
Precision del conjuto de entrenamiento: 49.77%
(1, 4)
[[15. 44.  0. 46.]]
(1, 5)
[1]
